<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/Capstone/MultiModal_phi_with_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


**Install dependencies**

In [ ]:
!pip install -U -quiet numpy transformers datasets tqdm matplotlib wandb torchmetrics torchinfo pillow pytorch-lightning peft bitsandbytes einops

**Imports**

In [3]:
!git clone "https://github.com/jyanivaddi/ERA_V1.git"
!git -C ERA_V1 pull
!git pull

Cloning into 'ERA_V1'...
remote: Enumerating objects: 2202, done.
remote: Counting objects: 100% (795/795), done.
remote: Compressing objects: 100% (433/433), done.
remote: Total 2202 (delta 405), reused 709 (delta 348), pack-reused 1407
Receiving objects: 100% (2202/2202), 276.88 MiB | 27.66 MiB/s, done.
Resolving deltas: 100% (1097/1097), done.
Updating files: 100% (263/263), done.
Already up to date.
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
import torchmetrics
import wandb
import os
import gc
import torch
import pickle
import json
import torchinfo
import torch.nn as nn
import numpy as np
import pytorch_lightning as pl
import torch.multiprocessing as mp
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from pathlib import Path
from typing import Union, List
from torch.cuda.amp import autocast
from matplotlib import pyplot as plt
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from transformers import AutoProcessor, CLIPVisionModel
from pytorch_lightning.callbacks import Callback
from peft import LoraConfig

from llava_instruct_dataset import LlavaFinetuneDataset, LlavaCollator, split_data_to_train_and_val
from model_finetune import LitMultiModalPhiFineTune, SimpleLinearBlock, model_summary